In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary

class PatchEmbed(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, out_channels, 
                            kernel_size=kernel_size, stride=stride)
        self.norm = nn.LayerNorm(out_channels)

    def forward(self, x):
        x = self.proj(x)  # (B, C, H, W)
        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)  # (B, L, C)
        x = self.norm(x)
        return x, H, W

class Mlp(nn.Module):
    def __init__(self, dim, mlp_ratio=4):
        super().__init__()
        hidden_dim = int(dim * mlp_ratio)
        self.fc1 = nn.Linear(dim, hidden_dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_dim, dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        return x

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, sr_ratio=1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q = nn.Linear(dim, dim)
        self.kv = nn.Linear(dim, dim * 2)
        self.sr_ratio = sr_ratio
        
        if sr_ratio > 1:
            self.sr = nn.Conv2d(dim, dim, 
                               kernel_size=sr_ratio, stride=sr_ratio)
            self.norm = nn.LayerNorm(dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        q = self.q(x).reshape(B, N, self.num_heads, 
                            self.head_dim).permute(0, 2, 1, 3)

        if self.sr_ratio > 1:
            x_ = x.permute(0, 2, 1).reshape(B, C, H, W)
            x_ = self.sr(x_).reshape(B, C, -1).permute(0, 2, 1)
            x_ = self.norm(x_)
            kv = self.kv(x_).reshape(B, -1, 2, self.num_heads, 
                                   self.head_dim).permute(2, 0, 3, 1, 4)
        else:
            kv = self.kv(x).reshape(B, -1, 2, self.num_heads, 
                                  self.head_dim).permute(2, 0, 3, 1, 4)
        k, v = kv[0], kv[1]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, sr_ratio=1, mlp_ratio=4):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = Attention(dim, num_heads, sr_ratio)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = Mlp(dim, mlp_ratio)

    def forward(self, x, H, W):
        x = x + self.attn(self.norm1(x), H, W)
        x = x + self.mlp(self.norm2(x))
        return x

class PVT224(nn.Module):
    def __init__(self, num_classes=200):
        super().__init__()
        # Stage configurations: [embed_dim, num_heads, sr_ratio, num_blocks]
        stage_cfgs = [
            [64, 1, 8, 2],   # Stage 1 (56x56)
            [128, 2, 4, 2],  # Stage 2 (28x28)
            [320, 5, 2, 2],  # Stage 3 (14x14)
            [512, 8, 1, 2],  # Stage 4 (7x7)
        ]
        
        # Initial patch embedding
        self.patch_embed1 = PatchEmbed(3, 64, kernel_size=4, stride=4)
        
        # Subsequent stages
        self.stages = nn.ModuleList()
        self.patch_embeds = nn.ModuleList()
        
        for i, (dim, heads, sr, blocks) in enumerate(stage_cfgs):
            if i > 0:
                self.patch_embeds.append(
                    PatchEmbed(stage_cfgs[i-1][0], dim, kernel_size=2, stride=2)
            stage = nn.Sequential(*[
                TransformerBlock(dim, heads, sr, mlp_ratio=4)
                for _ in range(blocks)
            ])
            self.stages.append(stage)
        
        # Classification head
        self.norm = nn.LayerNorm(stage_cfgs[-1][0])
        self.head = nn.Linear(stage_cfgs[-1][0], num_classes)

    def forward(self, x):
        B = x.shape[0]
        x, H, W = self.patch_embed1(x)
        
        for i, stage in enumerate(self.stages):
            if i > 0:
                x = x.permute(0, 2, 1).reshape(B, -1, H, W)
                x, H, W = self.patch_embeds[i-1](x)
            
            for blk in stage:
                x = blk(x, H, W)
        
        x = self.norm(x)
        x = x.mean(dim=1)
        x = self.head(x)
        return x

# Model Summary
model = PVT224()
summary(model, input_size=(1, 3, 224, 224))

# Calculate parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal Parameters: {total_params/1e6:.2f}M")